In [ ]:
import cv2
import mediapipe as mp

# MediaPipe solutions
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
mp_face = mp.solutions.face_detection

# Initialize webcam
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
) as hands, mp_face.FaceDetection(min_detection_confidence=0.7) as face_detection:

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Hand detection
        results_hands = hands.process(img_rgb)

        if results_hands.multi_hand_landmarks:
            for hand_landmarks in results_hands.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Face detection
        results_face = face_detection.process(img_rgb)
        if results_face.detections:
            for detection in results_face.detections:
                # Get bounding box
                bboxC = detection.location_data.relative_bounding_box
                h, w, c = frame.shape
                x, y, w_box, h_box = int(bboxC.xmin * w), int(bboxC.ymin * h), \
                                     int(bboxC.width * w), int(bboxC.height * h)
                cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)

        # Show result
        cv2.imshow("Hand + Face Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
